In [2]:
!pip install pycryptodome
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
import os

In [7]:
def generate_rsa_keys():
    # The cryptographic strength is primarily linked to the length of the RSA modulus n. In 2023, a sufficient length is deemed to be 3072 bits.
    key = RSA.generate(3072)
    private_key = key.export_key()
    public_key = key.publickey().export_key()
    return private_key, public_key

def save_file(filename, data):
    with open(filename, 'wb') as file:
        file.write(data)

def load_file(filename):
    with open(filename, 'rb') as file:
        return file.read()

def split_file(file_path, chunk_size):
    with open(file_path, 'rb') as f:
        chunk_num = 0
        while True:
            chunk = f.read(chunk_size)
            if not chunk:
                break
            yield chunk_num, chunk
            chunk_num += 1






In [4]:
def encrypt_chunk(chunk, aes_key):
    cipher = AES.new(aes_key, AES.MODE_EAX)
    ciphertext, tag = cipher.encrypt_and_digest(chunk)
    return cipher.nonce, tag, ciphertext

def decrypt_chunk(nonce, tag, ciphertext, aes_key):
    cipher = AES.new(aes_key, AES.MODE_EAX, nonce=nonce)
    chunk = cipher.decrypt_and_verify(ciphertext, tag)
    return chunk

def rsa_encrypt(data, public_key):
    rsa_key = RSA.import_key(public_key)
    cipher_rsa = PKCS1_OAEP.new(rsa_key)
    return cipher_rsa.encrypt(data)

def rsa_decrypt(encrypted_data, private_key):
    rsa_key = RSA.import_key(private_key)
    cipher_rsa = PKCS1_OAEP.new(rsa_key)
    return cipher_rsa.decrypt(encrypted_data)

In [12]:
def main():
    input_file = '/content/DemoText_Holmes.txt'
    output_file = '/content/Decrypted.txt'
    chunk_size = 2048 * 2048  # 4MB
    # Our input file is less THAN 4 MB, So we should get 1 chunk
    #Getting RSA keys
    private_key, public_key = generate_rsa_keys()
    save_file('private_key.pem', private_key)
    save_file('public_key.pem', public_key)

    # Split input files into chunks (with maximum memory of 1 MB)
    encrypted_chunks = []
    for chunk_num, chunk in split_file(input_file, chunk_size):
        aes_key = get_random_bytes(32)  # AES-256 key
        nonce, tag, ciphertext = encrypt_chunk(chunk, aes_key)
        encrypted_aes_key = rsa_encrypt(aes_key, public_key)
        encrypted_chunks.append((nonce, tag, ciphertext, encrypted_aes_key))

        # Save each encrypted chunk
        save_file(f'chunk_{chunk_num}.enc', nonce + tag + ciphertext)
        save_file(f'chunk_key_{chunk_num}.enc', encrypted_aes_key)

    # Decrypt and reassemble file
    with open(output_file, 'wb') as out_file:
        for chunk_num in range(len(encrypted_chunks)):
            # Load encrypted chunk and AES key
            encrypted_chunk = load_file(f'chunk_{chunk_num}.enc')
            encrypted_aes_key = load_file(f'chunk_key_{chunk_num}.enc')

            nonce = encrypted_chunk[:16]
            tag = encrypted_chunk[16:32]
            ciphertext = encrypted_chunk[32:]

            # Decrypt AES key
            aes_key = rsa_decrypt(encrypted_aes_key, private_key)

            # Decrypt chunk
            chunk = decrypt_chunk(nonce, tag, ciphertext, aes_key)

            # Write chunk to output file
            out_file.write(chunk)


In [13]:
if __name__ == '__main__':
    main()